# Aethr - Evaluation (Google Colab Web + Google Drive)

Evaluate trained YOLO11 weights stored in Google Drive.

In [ ]:
from pathlib import Path
import os
from google.colab import drive

drive.mount('/content/drive')

PROJECT_ROOT = Path('/content/aethr')
REPO_URL = 'https://github.com/VishwaJaya01/aethr.git'

if not PROJECT_ROOT.exists():
    !git clone {REPO_URL} {PROJECT_ROOT}

os.chdir(PROJECT_ROOT)
!git pull
!pip install -q -r requirements.txt
!nvidia-smi

print('PROJECT_ROOT:', PROJECT_ROOT)

In [ ]:
from pathlib import Path

DRIVE_WORKDIR = Path('/content/drive/MyDrive/aethr')
DRIVE_DATASET_DIR = DRIVE_WORKDIR / 'data' / 'raw' / 'solar-panel-defects-v1'
DATA_YAML = DRIVE_DATASET_DIR / 'data.yaml'
MODEL_PATH = DRIVE_WORKDIR / 'models' / 'best.pt'

if not DATA_YAML.exists():
    raise FileNotFoundError(f'Dataset YAML missing: {DATA_YAML}')
if not MODEL_PATH.exists():
    raise FileNotFoundError(f'Model missing: {MODEL_PATH}')

print('DATA_YAML :', DATA_YAML)
print('MODEL_PATH:', MODEL_PATH)

In [ ]:
from ultralytics import YOLO

model = YOLO(str(MODEL_PATH))
val_metrics = model.val(
    data=str(DATA_YAML),
    split='val',
    imgsz=640,
    device=0,
)

print('Validation metrics:')
print(val_metrics.results_dict)

In [ ]:
import yaml

data_cfg = yaml.safe_load(DATA_YAML.read_text(encoding='utf-8'))
has_test = bool(data_cfg.get('test'))

if has_test:
    test_metrics = model.val(
        data=str(DATA_YAML),
        split='test',
        imgsz=640,
        device=0,
    )
    print('Test metrics:')
    print(test_metrics.results_dict)
else:
    print('No test split declared in data.yaml. Skipping test evaluation.')